# Tutorial: Add swc
This notebook demonstrates how to load neuron traces from [swc](http://www.neuronland.org/NLMorphologyConverter/MorphologyFormats/SWC/Spec.html) files.

To run napari from a jupyter notebook, you need to instantiate a `QT GUI`. You must wait for the `QT GUI` to instantiate before creating the `Viewer`

In [1]:
%gui qt5
# Note that this Magics command needs to be run in a cell
# before any of the Napari objects are instantiated to
# ensure it has time to finish executing before they are
# called

import napari
import numpy as np
from skimage import io

## Loading entire neuron
To load the entire neuron from a swc, you need to specify the `swc_path`. By convention, vertex coordinates in the swc file are in micrometers (spatial units). Napari can read the swc file and convert the coordinates from spatial units to voxel units given the `spacing` (spatial units/voxel) and `origin` (spatial units).

Conversion from spatial to voxel units is done with the following equation:

$voxel = \frac{spatial - origin}{spacing}$

In [2]:
## show neuron G-002
# parameters
consen_neuron_path = '2018-08-01_G-002_consensus.swc'
spacing = np.array([0.29875923,0.3044159,0.98840415])
origin = np.array([70093.276,15071.596,29306.737])

In [3]:
## show entire neuron G-002
viewer = napari.Viewer(ndisplay=3)
viewer.add_swc(swc_path=consen_neuron_path, spacing=spacing, origin=origin, edge_color='white', name='G-002')

<Shapes layer 'G-002' at 0x1480a7160>

![screenshot](G-002_consensus.png)

Screenshot of entire neuron G-002 in napari. Neuron G-002 is from the [Mouselight](https://www.janelia.org/project-team/mouselight) data set.

## Loading sub-neuron

The image of the entire brain has dimensions of (33792, 25600, 13312) voxels. G-002 spans a sub-image of (7386, 9932, 5383) voxels. Both are too big to load in napari and overlay the neuron.
To circumvent this, we can crop out a smaller region of the neuron, load the sub-neuron, and load the corresponding sub-image.

In order to get a sub-neuron, we need to specify the `bounding_box` that will be used to crop the neuron. `bounding_box` is a length 2 tuple. The first element is one corner of the bounding box (inclusive) and the second element is the opposite corner of the bounding box (exclusive). Both corners are in voxel units.

In [4]:
## show sub-neuron from complex region of neuron G-002
start = np.array([15312,4400,6448])  # start corner of the bounding_box
end = np.array([15840,4800,6656])    # end corner of the bounding_box
viewer_comp = napari.Viewer(ndisplay=3)
viewer_comp.add_swc(swc_path=consen_neuron_path, spacing=spacing, origin=origin, bounding_box=(start,end), edge_color='blue', name='G-002_complex')

<Shapes layer 'G-002_complex' at 0x14a38afd0>

We will now overlay the corresponding sub-image with the sub-neuron. This sub-image was already cropped from the image of the full brain.

In [5]:
image_path = 'G-002_complex.tif'
img_comp = io.imread(image_path)
img_comp = np.swapaxes(img_comp,0,2) # this particular image is saved in (z,y,x) and so must have axes swapped to be (x,y,z)

viewer_comp.add_image(img_comp)

<Image layer 'Image' at 0x14a124278>

![screenshot](G-002_complex.png)

Screenshot of the sub-neuron loaded into napari